<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/public_container_at_ep5g.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

with open('demo_project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········


Install required packages and dependencies. Ignore the warnings.

In [ ]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.8/323.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.6/512.6 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

Import packages

In [ ]:
import json
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, get_available_publicips, get_segment_ids, get_worker_interfaces

In the next cell, we reserve the required equipment and resources to form an end to end experiment setup. We reserve EP5G network, one Advantech router and one worker to run the workloads.

In [ ]:
# worker reservation
worker_name = "worker-01"
worker_lease = show_reservation_byname(worker_name + "-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]

# ep5g reservation
ep5g_lease = show_reservation_byname("ep5g-lease")
ep5g_segment_id = str(list(get_segment_ids('ep5g').values())[0])
if not ep5g_lease:
    ep5g_lease = reserve(
        { "type":"network", "name": "ep5g", "net_name": "ep5g-vip", "segment_id": ep5g_segment_id, "duration": { "days":7, "hours":0 } }
    )

leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

2024-02-26 19:30:30.954 | INFO     | chi.expeca:reserve:167 - reserving ep5g
2024-02-26 19:30:33.540 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for ep5g-lease with id 1336affd-6aef-45f4-aa53-5870914b9c95 to become "ACTIVE"
2024-02-26 19:30:38.817 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 1336affd-6aef-45f4-aa53-5870914b9c95 is PENDING.
2024-02-26 19:30:44.100 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 1336affd-6aef-45f4-aa53-5870914b9c95 is PENDING.
2024-02-26 19:30:49.368 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 1336affd-6aef-45f4-aa53-5870914b9c95 is PENDING.
2024-02-26 19:30:54.653 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 1336affd-6aef-45f4-aa53-5870914b9c95 is PENDING.
2024-02-26 19:30:59.916 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 1336affd-6aef-45f4-aa53-5870914b9c95 is P

[
    {
        "name": "worker-01-lease",
        "id": "0461c496-d6bf-4fb3-a0b0-9ea2c1027be7",
        "reservation_id": "e650b561-3ab2-496a-95ce-e1e0db25ea6d",
        "status": "ACTIVE",
        "end_date": "2024-03-02T13:18:00.000000"
    },
    {
        "name": "ep5g-lease",
        "id": "1336affd-6aef-45f4-aa53-5870914b9c95",
        "reservation_id": "728feadf-98ba-4c7d-b7a1-6e6116488191",
        "status": "ACTIVE",
        "end_date": "2024-03-04T19:30:00.000000"
    }
]


In the following section we setup the networking equipment for ep5g (refer to [here](https://kth-expeca.gitbook.io/testbedconfig/enroll/enroll-network-segments/ep5g) for more info). It contains creation of an edge-net, a router, and some interfaces on the router and routes.

In [ ]:
# create edge-net
edgenet = chi.network.create_network("edge-net")
chi.network.create_subnet("edge-net-subnet",edgenet["id"],"10.70.70.0/24",gateway_ip="10.70.70.1",enable_dhcp=False)
logger.success("edge-net is created.")

# create ep5g-vip-router
router = chi.network.create_router("ep5g-vip-router","public")
logger.success("ep5g-vip-router router is created.")
logger.info(f"{json.dumps(router,indent=4)}")

# connect ep5g-vip-net to ep5g-vip-router
ep5gnet = chi.network.get_network("ep5g-vip-net")
portadd = chi.network.add_subnet_to_router(router["id"],ep5gnet["subnets"][0])
logger.success("An interface on ep5g-vip-net is added to the router")

# create edge-net to ep5g-vip-router
edgenet = chi.network.get_network("edge-net")
portadd = chi.network.add_subnet_to_router(router["id"],edgenet["subnets"][0])
logger.success("An interface on edge-net is added to the router")

# add ep5g route to ep5g-vip-router
routeadd = chi.network.add_route_to_router(router["id"],"172.16.0.0/16","10.30.111.10")
logger.success("the route added to the router")

2024-02-26 19:32:29.074 | SUCCESS  | __main__:<cell line: 4>:4 - edge-net is created.
2024-02-26 19:32:32.005 | SUCCESS  | __main__:<cell line: 8>:8 - ep5g-vip-router router is created.
2024-02-26 19:32:32.007 | INFO     | __main__:<cell line: 9>:9 - {
    "id": "57d59197-de50-4336-8fe3-e200ce92414f",
    "name": "ep5g-vip-router",
    "tenant_id": "2379270948e848d99073172ae425701d",
    "admin_state_up": true,
    "status": "ACTIVE",
    "external_gateway_info": {
        "network_id": "717b5f2b-069e-4868-a24d-91a4ae3ad002",
        "external_fixed_ips": [
            {
                "subnet_id": "53d03ffd-0d49-4f9a-88a3-a30d69fe4827",
                "ip_address": "130.237.11.108"
            }
        ],
        "enable_snat": true
    },
    "description": "",
    "availability_zones": [],
    "availability_zone_hints": [],
    "routes": [],
    "flavor_id": null,
    "tags": [],
    "created_at": "2024-02-26T19:32:29Z",
    "updated_at": "2024-02-26T19:32:30Z",
    "revision_num

Now we start running the container.

In [ ]:
# check public IPs and select one
available_pub_ips = get_available_publicips()
pub_ip = available_pub_ips[0]
logger.info(f"Available public ips: {available_pub_ips}.")
logger.info(f"We choose {pub_ip} for this container.")

# check available interfaces of the worker
interfaces = list(get_worker_interfaces(worker_name).values())[0]
available_ifs = []
for interface in interfaces.keys():
  if len(interfaces[interface]['connections']) == 0:
    available_ifs.append(interface)
logger.info(f"Available interfaces on {worker_name}: {available_ifs}")
if len(available_ifs) < 2:
  logger.info(f"{json.dumps(interfaces, indent=4)}")
  raise Exception(f"Did not find enough interfaces on {worker_name}")

# run container
publicnet = chi.network.get_network("serverpublic")
edgenet = chi.network.get_network("edge-net")
container_name = "my-server-node"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/sshd-image",
    reservation_id = worker_reservation_id,
    environment = {
        "DNS_IP":"8.8.8.8",
        "GATEWAY_IP":"130.237.11.97",
        "PASS":"expeca"
    },
    mounts = [],
    nets = [
        { "network" : publicnet['id'] },
        { "network" : edgenet['id'] },
    ],
    labels = {
        "networks.1.interface":available_ifs[0],
        "networks.1.ip":pub_ip+"/27",
        "networks.1.gateway":"130.237.11.97",
        "networks.2.interface":available_ifs[1],
        "networks.2.ip":"10.70.70.3/24",
        "networks.2.routes":"172.16.0.0/16-10.70.70.1",
        "capabilities.privileged":"true",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container, reachable at {pub_ip}")

2024-02-26 19:36:45.757 | SUCCESS  | __main__:<cell line: 29>:29 - created samie-server-node container.


Now you can ssh into the container via
```
ssh root@130.237.11.115
```
From anywhere. Also, you can run a remote command here. The result will be printed on the logs page in Chameleon.

In [ ]:
command = "ip route show"
result = chi.container.execute(
    container_ref=container_name,
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2024-02-24 17:06:10.520 | INFO     | __main__:<cell line: 6>:6 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}
